In [1]:
#!pip install fastai

In [3]:

import gc 
import pandas as pd
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
%matplotlib inline



In [4]:
PATH = "/Users/krishna/Documents/Projects/smart mirror phase 2/"
TRAINING_PATH = "Category and Attribute Prediction Benchmark/Anno_fine/new files/"
FNAME = "train_labels.csv"
#TEST_PATH = "clothes_categories/test_labels.csv"
#CUSTOM_DATASET_PATH = "clothes_categories/test_labels_phone.csv"
#CLASSES_PATH = "clothes_categories/classes.txt"

In [5]:
train_df = pd.read_csv(PATH+TRAINING_PATH+FNAME)
train_df.head(5)

,image_name,category_name
0,Sweet_Crochet_Blouse/img_00000070.jpg,Blouse
1,Classic_Pencil_Skirt/img_00000010.jpg,Skirt
2,Strapless_Diamond_Print_Dress/img_00000038.jpg,Dress
3,Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010.jpg,Jeans
4,Zippered_Single-Button_Blazer/img_00000078.jpg,Blazer


In [5]:
# from PIL import Image
# import time

# for i in range(20):
#     dire = PATH+train_df.iloc[i,0]
#     img = Image.open(dire)
#     img.show()
#     time.sleep(0.1)


In [6]:

# data = ImageDataLoaders.from_csv(PATH+"img/",csv_fname=FNAME,
#                                  item_tfms=Resize(1),
#                                  batch_tfms=aug_transforms(size=1, min_scale=0.9),
#                                  valid_pct=0.1,
#                                  splitter=RandomSplitter(seed=42), #seed=42
#                                  num_workers=0)


In [7]:
from keras.preprocessing.image import ImageDataGenerator

imageSize=(224,224)
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=PATH+TRAINING_PATH+"train_category/",target_size=imageSize)
ttdata = ImageDataGenerator()
testdata = ttdata.flow_from_directory(directory=PATH+TRAINING_PATH+"test_category/",target_size=imageSize)

Found 14000 images belonging to 51 classes.
Found 4000 images belonging to 51 classes.


In [20]:

from keras.applications.vgg16 import VGG16
#model = VGG16(include_top=False,input_shape=(32,32,3),weights='imagenet',classifier_activation='softmax',classes=20)
imageShape=(224,224,3)
imageSize=(224,224)
vgg = VGG16(input_shape=imageShape, weights='imagenet', include_top=False, pooling='avg')
vgg.trainable = False
model = Sequential()
model.add(vgg)

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=51, activation="softmax"))


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 4096)              2101248   
                                                                 
 dense_4 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_5 (Dense)             (None, 51)                208947    
                                                                 
Total params: 33,806,195
Trainable params: 19,091,507
Non-trainable params: 14,714,688
_________________________________________________________________


In [31]:
from keras.optimizers import SGD
opt = SGD(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [11]:
# hist = model.load_weights("/Users/krishna/Documents/Projects/Smart_Mirror/vgg16_1.h5")
#

In [32]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("/Users/krishna/Documents/Projects/smart mirror phase 2/vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
# hist = model.fit(generator=traindata, validation_data= testdata, validation_steps=10,epochs=20,callbacks=[checkpoint,early])

In [34]:
hist = model.fit(
    traindata,
    validation_data=testdata,
    epochs=20,
    verbose="auto",
    callbacks=[checkpoint,early],
    validation_steps=10,
)

Epoch 1/20
 27/438 [>.............................] - ETA: 24:05 - loss: 2.9776 - accuracy: 0.2523

KeyboardInterrupt: 